In [2]:
import requests
import json
#from collections.abc import Mapping
import itertools

In [3]:
API = '4wJ8FlXRXOYo0Fn1JbsOO9zz4PHUvKU1aKCvVsZMpWkw1tkD0dq12U2uYGXbZZEg'
url = "https://statement.sandbox.bnk.to/v1/statement-init"

payload = json.dumps({
  "external_id": "Test-01010103",
  "country": "ID",
  "bank_codes": [
    "DUMMY_BANK_PERSONAL"
  ],
  "organization_display_name": "TONGDUN"
})

headers = {
  'Content-Type': 'application/json',
  'x-api-key': API
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"statement_id":"69058eee-cbba-4ae2-b838-70a2d1a2c15e", "redirect_uri":"https://tap.sandbox.brank.as?app=statement&statement_id=69058eee-cbba-4ae2-b838-70a2d1a2c15e&country=ID&organization_display_name=TONGDUN&bank_codes=MjYx"}


In [4]:
data = json.loads(response.text)
data_id = data['statement_id']
data_uri = data['redirect_uri']
print(data_uri)

https://tap.sandbox.brank.as?app=statement&statement_id=69058eee-cbba-4ae2-b838-70a2d1a2c15e&country=ID&organization_display_name=TONGDUN&bank_codes=MjYx


In [5]:
url2 = "http://statement.sandbox.bnk.to/v1/statements?statement_ids=" + data_id

headers = {
  'Content-Type': 'application/json',
  'x-api-key': API
}

response2 = requests.request("GET", url2, headers=headers)

print(response2.text)

{"statements":[{"status":"COMPLETED", "statement_id":"69058eee-cbba-4ae2-b838-70a2d1a2c15e", "bank_code":"DUMMY_BANK_PERSONAL", "start_date":"2018-10-31T17:00:00Z", "end_date":"2021-10-15T06:21:14Z", "account_statements":[{"account":{"type":"SAVINGS", "account_number":"0007-123-007", "balance":{"cur":"IDR", "num":"50000000000", "decimal":{"places":2, "num":"500000000.00"}}, "holder_name":"TEST USER", "total_amount_credited":{"cur":"IDR", "num":"10500000", "decimal":{"places":2, "num":"105000.00"}}, "total_amount_debited":{"cur":"IDR", "num":"10500000", "decimal":{"places":2, "num":"105000.00"}}, "credit_transaction_count":2, "debit_transaction_count":1}, "transactions":[{"date":"2018-10-31T17:00:00Z", "type":"DEBIT", "status":"SUCCESS", "amount":{"cur":"IDR", "num":"10500000", "decimal":{"places":2, "num":"105000.00"}}, "descriptor":"Payment for merchant A", "balance":{"cur":"IDR", "num":"50000000000", "decimal":{"places":2, "num":"500000000.00"}}}, {"date":"2018-11-01T17:00:00Z", "typ

In [35]:
statement_dict = json.loads(response2.text)
for lst in statement_dict['statements']:
    for item in lst:
        if item == 'account_statements': 
            print("######         ACCOUNT STATEMENT         ######")
            for lst in statement_dict['statements']:
                for item in lst['account_statements']:
                    if list(item.keys()) == ['account', 'transactions']:
                        print("######         ACCOUNT                   ######")
                        for detail in item['account']:
                            if detail not in ('balance', 'total_amount_credited', 'total_amount_debited'):
                                print(detail, ':', item['account'][detail])
                            elif detail in ('balance', 'total_amount_credited', 'total_amount_debited'):
                                print(detail, ':', item['account'][detail]['decimal']['num'])
                        for transaction in item['transactions']:
                            print('\n')
                            for t in transaction:
                                if t not in ('amount', 'balance'):
                                    print(t, ':', transaction[t])
                                elif t in ('amount', 'balance'):
                                    print(t, ':', transaction[t]['decimal']['num'])
                    elif list(item.keys()) == ['account']:
                        print('\n')
                        for detail in item['account']:
                            if detail not in ('balance', 'total_amount_credited', 'total_amount_debited'):
                                print(detail, ':', item['account'][detail])
                            elif detail in ('balance', 'total_amount_credited', 'total_amount_debited'):
                                print(detail, ':', item['account'][detail]['decimal']['num'])
                        print('\n')
        elif item not in 'account_statements':
            print(item, ':', lst[item], '\n')

status : COMPLETED 

statement_id : 69058eee-cbba-4ae2-b838-70a2d1a2c15e 

bank_code : DUMMY_BANK_PERSONAL 

start_date : 2018-10-31T17:00:00Z 

end_date : 2021-10-15T06:21:14Z 

######         ACCOUNT STATEMENT         ######
######         ACCOUNT                   ######
type : SAVINGS
account_number : 0007-123-007
balance : 500000000.00
holder_name : TEST USER
total_amount_credited : 105000.00
total_amount_debited : 105000.00
credit_transaction_count : 2
debit_transaction_count : 1


date : 2018-10-31T17:00:00Z
type : DEBIT
status : SUCCESS
amount : 105000.00
descriptor : Payment for merchant A
balance : 500000000.00


date : 2018-11-01T17:00:00Z
type : CREDIT
status : SUCCESS
amount : 100000.00
descriptor : Payment from merchant B
balance : 500105000.00


date : 2018-11-02T17:00:00Z
type : CREDIT
status : SUCCESS
amount : 5000.00
descriptor : Payment from merchant C
balance : 500005000.00


type : CHECKING
account_number : 0007-321-456
balance : 234500000.00
holder_name : TEST USE

In [109]:
statement_dict = json.loads(response2.text)

print('statement_id : ' + statement_dict['statements'][0]['statement_id'])
print('bank_code : ' + statement_dict['statements'][0]['bank_code'])
print('statement_start_date : ' + statement_dict['statements'][0]['start_date'])
print('statement_end_date : ' + statement_dict['statements'][0]['end_date'])


statement_id : e2ef391d-646b-4410-b860-951b024f8206
bank_code : DUMMY_BANK_PERSONAL
statement_start_date : 2018-10-31T17:00:00Z
statement_end_date : 2021-10-14T04:05:40Z


In [ ]:
#def layers (statement_dict):
#    if isinstance (statement_dict, Mapping):
#        yield statement_dict.keys()
#        yield from layers(next(statement_dict.values()))